In [0]:
##install pytorch in colab
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
!pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl

In [2]:
##mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import csv
import json
import pandas as pd
import torch
import time
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn.init as init

In [0]:
#dataset
class load_dataset(Dataset):
    def __init__(self, data_path):
        self.samples = pd.read_csv(data_path).values

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx,:-1] ,self.samples[idx,-1]
      
###Model

class MultilayerPerceptron(torch.nn.Module):

    def __init__(self):
        super(MultilayerPerceptron, self).__init__()
        
        self.hidden = torch.nn.ModuleList()
        self.hidden.append(torch.nn.Linear(num_features, hidden_nodes_list[0]))
        for k in range(num_hidden_layes-1):
            self.hidden.append(torch.nn.Linear(hidden_nodes_list[k], hidden_nodes_list[k+1]))
             
        ### Output layer
        self.hidden.append(torch.nn.Linear(hidden_nodes_list[num_hidden_layes-1], num_classes))
        
    def forward(self, x):
        out = x
        for layer in self.hidden[:-1]:
          out = layer(out)
          out = F.relu(out)

        logits = self.hidden[-1](out)
        probas = F.log_softmax(logits, dim=1)
        return logits, probas
      
#weight initialization function
def init_weights(m):
  if isinstance(m, torch.nn.Linear):
    if initialisation_method=='xavier':
      init.xavier_uniform_(m.weight)
    if initialisation_method=='he':
      init.kaiming_uniform_(m.weight)
    m.bias.data.fill_(0.01)
    
#compute accuracy function
def compute_accuracy(net, data_loader):
    net.eval()
    cost, correct_pred, num_examples = 0, 0, 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.float().to(device)
            targets = targets.long().to(device)
            logits, probas = net(features)
            cost += F.cross_entropy(logits, targets) * targets.size(0)
            _, predicted_labels = torch.max(probas, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
        return cost/num_examples , correct_pred.float()/num_examples * 100

      

In [0]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#data
points_dim = 32
radius_of_sphere = 1
radius_of_shell = 1.6
thickness_of_shell = 1
csv_path = '/content/drive/My Drive/DL_exp/data/train_sp'+ str(radius_of_sphere) + '_sh'+ str(radius_of_shell) + '_t' + str(thickness_of_shell)+ '_' + str(points_dim) + 'dim.csv'     

dataset = load_dataset(csv_path)
size = len(dataset)


#architecture
num_features  = next(iter(dataset))[0].shape[0]        # Input data dimention
hidden_nodes_list   = [64]                              # List of number of nodes at each hidden layer
num_hidden_layes = len(hidden_nodes_list)              # The number of hidden layers
num_classes   = 2                                      # The number of output classes. In this case, 0 and 1

# 'xavier' : Xavier Initialisation
# 'he' : He Initialisation
initialisation_method = 'xavier'

# 'sgd' : SGD (lr) 
# 'sgdwm' : SGD with Momentum (lr, momentum)
# 'adagrad' : AdaGrad
# 'adam' : Adam
# 'ngd' : Natural gradient descent
# 'l1' : L1 Regularisation
# 'l2' : L2 Regularisation
# 'pathnorm' : PathNorm Regularisation
# 'spectralnorm' : Spectral norm Regularisation
optimisation_method = 'sgdwm'  

# Hyperparameters
random_seed = 400
learning_rate = 0.05
num_epochs = 100
batch_size = size

##load dataset
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

torch.manual_seed(random_seed)
model = MultilayerPerceptron()
model.apply(init_weights)
model = model.to(device)

if optimisation_method=='sgd' or optimisation_method=='ngd':
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
if optimisation_method=='sgdwm':
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
if optimisation_method=='adagrad':
  optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)
if optimisation_method=='adam':
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08)

epoch_loss = []
### training of a model
start_time = time.time()
epoch = 0

with torch.set_grad_enabled(False):
  cost,best_acc = compute_accuracy(model, train_dataloader)
  print('Epoch: %03d | Accuracy: %.2f%% | Cost: %.4f' % (epoch,best_acc,cost))
epoch_loss.append([0,11])

# for epoch in range(num_epochs):
count = 1
prev_acc=best_acc
best_epoch = epoch
best_cost = cost

while True:
    model.train()
    for batch_idx, (features, targets) in enumerate(train_dataloader):
        features = features.float().to(device)
        targets = targets.long().to(device)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
    epoch+=1
    
    with torch.set_grad_enabled(False):
        cost,acc = compute_accuracy(model, train_dataloader)
        print('Epoch: %03d | Accuracy: %.2f%% | Cost: %.4f' % (epoch,acc,cost))
        
    if prev_acc==acc:
      count+=1
    else:
      prev_acc=acc
      count=1
    if (epoch>50 and best_acc-acc>=5) or count==20 or epoch==400:
      break
    if acc>best_acc:
      best_acc=acc
      best_epoch = epoch
      best_cost = cost
    
#     print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

In [5]:
### training of multiple models and save the result in JSON file
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

start_time = time.time()
radius_of_sphere = 1
radius_of_shell = 2
thickness_of_shell = 1

with open('/content/drive/My Drive/DL_exp/result/loss.json', mode='r') as readjson:
  try: 
    loss = json.loads(readjson.read())
  except ValueError: 
    loss = {}
          
dimentions = [2,4,8,16,32,64]          #list of dimention of data
for dim in dimentions:
  csv_path = '/content/drive/My Drive/DL_exp/uniform_data/train_sp'+ str(radius_of_sphere) + '_sh'+ str(radius_of_shell) + '_t' + str(thickness_of_shell)+ '_' + str(dim) + 'dim.csv'
  dataset = load_dataset(csv_path)
  size = len(dataset)
  train_dataloader = DataLoader(dataset, batch_size=size, shuffle=True)


  tmp_h = [2,4,6,8,12,16,24,32,48,64,128,256]             #list of number of nodes in hidden layer
  tmp_i = [100,200,300,400,500,600,700,800,900,1000]      #list of random seed for weight initialization

  for t_h in tmp_h:
    for t_i in tmp_i:

      #architecture
      num_features  = next(iter(dataset))[0].shape[0]        # Input data dimention
      hidden_nodes_list   = [t_h]                              # List of number of nodes at each hidden layer
      num_hidden_layes = len(hidden_nodes_list)              # The number of hidden layers
      num_classes   = 2                                      # The number of output classes. In this case, 0 and 1

      initialisation_method = 'xavier'
      optimisation_method = 'sgdwm'  

      # Hyperparameters
      random_seed = t_i
      learning_rate = 0.05
            
      torch.manual_seed(random_seed)
      model = MultilayerPerceptron()
      model.apply(init_weights)
      model = model.to(device)

      optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
      model_loss = []
      epoch = 0

      with torch.set_grad_enabled(False):
        cost,best_acc = compute_accuracy(model, train_dataloader)
      model_loss.append([epoch,cost.data.tolist()])
      # for epoch in range(num_epochs):
      count = 1
      prev_acc=best_acc
      best_epoch = epoch
      best_cost = cost

      while True:
          model.train()
          for batch_idx, (features, targets) in enumerate(train_dataloader):

              features = features.float().to(device)
              targets = targets.long().to(device)

              ### FORWARD AND BACK PROP
              logits, probas = model(features)
              cost = F.cross_entropy(logits, targets)
              optimizer.zero_grad()

              cost.backward()

              ### UPDATE MODEL PARAMETERS
              optimizer.step()


          with torch.set_grad_enabled(False):
              cost,acc = compute_accuracy(model, train_dataloader)
              
          epoch+=1
          model_loss.append([epoch,cost.data.tolist()])
          if prev_acc==acc:
            count+=1
          else:
            prev_acc=acc
            count=1
          if (epoch>50 and best_acc-acc>=5) or count==20 or epoch==400:
            break
          if acc>best_acc:
            best_acc=acc
            best_epoch = epoch
            best_cost = cost
 
      print('Epoch: %03d | Accuracy: %.2f%% | Cost: %.4f' % (best_epoch,best_acc,best_cost))
  
      if len(hidden_nodes_list)==1:
        loss.update({'sh'+ str(radius_of_shell)+'dim'+str(dim)+'h'+str(t_h)+'i'+str(t_i):model_loss})
      else:
        loss.update({'sh'+ str(radius_of_shell)+'dim'+str(dim)+'h'+str(t_h)+'i'+str(t_i)+'l2':model_loss})
    
      with open('/content/drive/My Drive/DL_exp/result/uniform_result.json', mode='r') as readjson:
        try: 
          result = json.loads(readjson.read())
        except ValueError: 
          result = []  
      
        
      with open('/content/drive/My Drive/DL_exp/result/uniform_result.json', mode='w') as feedjson:
        entry = {'sphere_radius': 1, 'shell_radius': radius_of_shell, 'thickness' : 1, 'dimension':dim, 'n_nodes':hidden_nodes_list, 'random_seed':t_i, 'epoch':best_epoch, 'accuracy':best_acc.data.tolist(), 'cost':best_cost.data.tolist()}
        result.append(entry)
        json.dump(result, feedjson)

with open('/content/drive/My Drive/DL_exp/result/loss.json', mode='w') as feedjson:
  json.dump(loss, feedjson)


Epoch: 078 | Accuracy: 86.20% | Cost: 0.3343
Epoch: 090 | Accuracy: 89.60% | Cost: 0.2856
Epoch: 045 | Accuracy: 63.20% | Cost: 0.6925
Epoch: 039 | Accuracy: 64.80% | Cost: 0.6756
Epoch: 195 | Accuracy: 89.60% | Cost: 0.2708
Epoch: 069 | Accuracy: 84.80% | Cost: 0.3917
Epoch: 206 | Accuracy: 90.00% | Cost: 0.2657
Epoch: 097 | Accuracy: 69.80% | Cost: 0.5406
Epoch: 143 | Accuracy: 89.40% | Cost: 0.3071
Epoch: 181 | Accuracy: 86.20% | Cost: 0.3069
Epoch: 122 | Accuracy: 100.00% | Cost: 0.0849
Epoch: 074 | Accuracy: 100.00% | Cost: 0.1490
Epoch: 092 | Accuracy: 99.80% | Cost: 0.0968
Epoch: 070 | Accuracy: 100.00% | Cost: 0.1470
Epoch: 051 | Accuracy: 100.00% | Cost: 0.1495
Epoch: 059 | Accuracy: 100.00% | Cost: 0.1453
Epoch: 122 | Accuracy: 100.00% | Cost: 0.0696
Epoch: 116 | Accuracy: 100.00% | Cost: 0.0749
Epoch: 073 | Accuracy: 100.00% | Cost: 0.1150
Epoch: 046 | Accuracy: 100.00% | Cost: 0.1723
Epoch: 055 | Accuracy: 100.00% | Cost: 0.1254
Epoch: 034 | Accuracy: 100.00% | Cost: 0.2516